<font size=12px>Python script to test import in openLCA and automatic modelling dataset </font>

This script blablabla .... 

<font size=2px>First, the import of libs, setup of the IPC connection with the OpenLCA database. The database is expected to feature : 
* Ecoinvent datasets (cut-off modelling) 
* ESA datasets
* Nyx Earth Life cycle modelling datasets

The interfaces between the various DB are set of specific process id and the generation of all the process and their links + A list of parameters will be updated as well</font>

![Example Image](PBS_High_Level_generic.png)


In [20]:
#### Lib Imports _ Please select a kernel with the latest version of olca IPC API  [TBVerified and replicated]
import pandas as pd
import numpy as np
import json
from tabulate import tabulate
import random as rand
from datetime import datetime, timedelta
import statistics as stats
import matplotlib as mp
import matplotlib.pyplot as plt
import scipy.stats as stat
#import olca as ipc (Old_version of Ipc API)
import olca_ipc as ipc
import olca_schema as o



from typing import Callable
###########################################################
client = ipc.Client(8080)
# OpenLCA IPC server Port
###########################################################

In [21]:
### Verification of all the elements included 
# Ecoinvent (Stats)

#ESA (Stats)

#life Cycle modelling 
print("\nProduct Systems :")
for group_PS in client.get_all(o.ProductSystem):
    print(f"{group_PS.name} : {group_PS.target_amount} {group_PS.target_unit.name} ___ Version : {group_PS.version} Last Updt : {group_PS.last_change}" )

print("\nPBS :")
for group_PBS in client.get_descriptors(o.Process):
    if group_PBS.category == "Nyx Earth PBS" : 
        print(f"{group_PBS.name} {group_PBS} :: {group_PBS.id}")


print("\nDB Processes :")
for group_Proc in client.get_descriptors(o.Process):
    if group_Proc.category.startswith("Database") : 
        print(f"{group_Proc.name} {group_Proc} :: {group_Proc.id}")

print("\nFlow :")
for group_flow in client.get_descriptors(o.Flow):
    if group_flow.category == "Nyx Earth PBS" : 
        print(f"{group_flow.name} in {group_flow.ref_unit} :: {group_flow.id}")



Product Systems :
Power System : 1.0 Item(s) ___ Version : 00.00.000 Last Updt : 2023-11-27T15:41:26.156Z
Power System Assembly & Qualification : 1.0 Item(s) ___ Version : 00.00.000 Last Updt : 2023-11-28T08:05:13.641Z

PBS :
Battery Ref(id='ed53308d-d0fd-4d4b-85ba-896f2e1afbe0', category='Nyx Earth PBS', description=None, flow_type=<FlowType.PRODUCT_FLOW: 'PRODUCT_FLOW'>, location=None, name='Battery', process_type=<ProcessType.UNIT_PROCESS: 'UNIT_PROCESS'>, ref_unit=None, ref_type=<RefType.Process: 'Process'>) :: ed53308d-d0fd-4d4b-85ba-896f2e1afbe0
Control Unit Ref(id='58fd6ee5-43f8-4814-b188-d6b05a36158b', category='Nyx Earth PBS', description=None, flow_type=<FlowType.PRODUCT_FLOW: 'PRODUCT_FLOW'>, location=None, name='Control Unit', process_type=<ProcessType.UNIT_PROCESS: 'UNIT_PROCESS'>, ref_unit=None, ref_type=<RefType.Process: 'Process'>) :: 58fd6ee5-43f8-4814-b188-d6b05a36158b
Power System Ref(id='4fc1a016-3532-4144-b6f5-e38165d59f57', category='Nyx Earth PBS', description=N

In [22]:
###########################################################
client = ipc.Client(8080)
# OpenLCA IPC server Port
###########################################################
### Prevent duplicata 
# ID Dictionary
Dic_ID_process = {}
for process in client.get_descriptors(o.Process):
    if process.category == "Nyx Earth PBS":
        Dic_ID_process[process.name] = process.id

Dic_ID_flow = {}
for flow in client.get_descriptors(o.Flow):
    if flow.category == "Nyx Earth PBS":
        Dic_ID_flow[flow.name] = flow.id

#### Extracting Units
# 

Extracted_flow_units = []
for units in client.get_all(o.FlowProperty) : 
    Extracted_flow_units.append(units.name)
Extracted_flow_units.sort()

### Read Excel Database
# 
# Read the excel file 
Process_DB = pd.read_excel('process.xlsx')
Process_DB['Name'].ffill(inplace=True)
Process_DB['Description'].ffill(inplace=True)
Process_DB['Tags'].ffill(inplace=True)
Process_DB['Ex.amount'] = Process_DB['Ex.amount'].astype(int)

Process_list = Process_DB.to_dict(orient='records')
#Process_list = list(zip(Process_DB['Name'], Process_DB['Description'],Process_DB['Tags'], Process_DB['Ex.name'], Process_DB['Ex.amount'],Process_DB['Ex.units'],Process_DB['Ex.type'],Process_DB['Ex.provider'])) 
print (tabulate(Process_DB,headers="keys"))


    Name                                   Description                                    Tags                          Ex.name                 Ex.amount  Ex.units                         Ex.type    Ex.provider                  Ex.uncertainty
--  -------------------------------------  ---------------------------------------------  ----------------------------  --------------------  -----------  -------------------------------  ---------  -------------------------  ----------------
 0  Battery                                A battery dummy                                Spacecraft,Power, Equipment   Cells                         -25  Number of items                  Product    Cells_Europe                          0.625
 1  Battery                                A battery dummy                                Spacecraft,Power, Equipment   Al _ Part                     -10  Mass                             Product    Al_Milled_Europe                      0.25
 2  Battery                  

First we export the flows 

In [23]:
# Allow to create a flow with out duplicata from dico in pbs with the name 
def create_flow(
    name: str, units: str, fn: Callable[[str, o.FlowProperty], o.Flow]
) -> o.Flow:
        
    prop = client.get(o.FlowProperty,name = units)
    if prop is None:
        raise ValueError(f"Invalid FlowProperty name: {units} \n Valid Unit Names are {Extracted_flow_units}")
    
    flow = fn(name, prop)
    M,m,v = map(int, flow.version.split('.'))
    # Upload the flow in the right category & prevent duplicata
    flow.category= "Nyx Earth PBS"
    if flow.name in Dic_ID_flow:
        flow.id = Dic_ID_flow[flow.name]
        M,m,v = map(int, client.get(o.Flow,name = name).version.split('.'))
        print (f"Updated to openLCA flows : {flow.name} :: {flow.id}")        
    else : 
        print (f"Exported to openLCA flows : {flow.name} :: {flow.id}")
      
    flow.version = f"{M:02d}.{m:02d}.{(v+1):03d}"
    flow.last_change = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    flow.description= f"Flow representing the produced {name}"

    client.put(flow)
    Dic_ID_flow[flow.name] = flow.id
    return flow


# Read the excel file 
for Flow in Process_list : 
    if Flow['Ex.type'] == "Product" : 
        create_flow(Flow['Ex.name'],Flow['Ex.units'], o.new_product)
    if Flow['Ex.type'] == "Waste" : 
        create_flow(Flow['Ex.name'],Flow['Ex.units'], o.new_waste)
    if Flow['Ex.type'] == "Env." :
        None
    else :
        "Error, Wrong Flow Type"


Updated to openLCA flows : Cells :: 02a06ce4-9656-421e-89ff-9799d82dce27
Updated to openLCA flows : Al _ Part :: 55ed0bfc-6184-4f38-9858-d046557071c1
Updated to openLCA flows : Battery :: e9664422-46f5-46ad-a3a0-6cdfbc5e7b4a
Updated to openLCA flows : PCB_Prod_Asia :: c9f9ee97-4f02-414c-9ece-fc814ab0f923
Updated to openLCA flows : Al _ Part :: 55ed0bfc-6184-4f38-9858-d046557071c1
Updated to openLCA flows : Control Unit :: 5ced102e-ba7f-4bb7-b264-462fc97542e0
Updated to openLCA flows : Harness :: ad4c3464-7541-4b37-9b69-27869a777382
Updated to openLCA flows : Harness Syst :: 03be4ebb-4cb6-46c5-8c59-93ceb49cc439
Updated to openLCA flows : Battery :: e9664422-46f5-46ad-a3a0-6cdfbc5e7b4a
Updated to openLCA flows : Control Unit :: 5ced102e-ba7f-4bb7-b264-462fc97542e0
Updated to openLCA flows : Harness Syst :: 03be4ebb-4cb6-46c5-8c59-93ceb49cc439
Updated to openLCA flows : Lorry EU Transport :: a19b3382-68e8-460c-9492-76d124131e90
Updated to openLCA flows : AIT Planegg time :: f2117d51-4af9-

However, you can set the default provider also via olca-ipc. The important thing here is to differentiate between flows and exchanges. Exchanges are the in- and outputs of processes. Flows are "part of" exchanges, and you can set the default provider of any exchange upon creation, for example:

exchange.default_provider = client.get(olca.Process, ##process.ref##)

Of course, you need to change ##process.ref##, e.g. to the UUID of the providing process.

Then all the process 

In [24]:
def create_process(name: str, description:str,tags:str, Ex_name: str, Ex_amount:int, Ex_Prov:str,Ex_std:int, Progress_list: list) -> o.Process:
    if name not in Progress_list : 
        process = o.new_process(name)
    else : 
        process = client.get(o.Process,name=name)
   
    process.description = description
    process.tags =tags.split(", ")
    
    exchange = client.get(o.Flow,name=Ex_name)
    if Ex_amount > 0:
        e=o.new_output(process, exchange, Ex_amount)
    if Ex_amount < 0:
        e=o.new_input(process, exchange, abs(Ex_amount))
    if Ex_Prov == "REF" :
        e.is_quantitative_reference = True
    else : 
        provider =client.get(o.Process,name=Ex_Prov) 
        if provider is None:
            raise ValueError(f"Invalid provider name: {Ex_Prov} \n") 

    e.uncertainty = o.Uncertainty(distribution_type=o.UncertaintyType.NORMAL_DISTRIBUTION,mean=abs(Ex_amount),sd=Ex_std)
 
    #    e.ProviderLinking(provider)

    M,m,v = map(int, process.version.split('.'))
    # Upload the flow in the right category & prevent duplicata
    process.category= "Nyx Earth PBS"
    if name in Dic_ID_process:
        process.id = Dic_ID_process[process.name]
        M,m,v = map(int, client.get(o.Process,name = name).version.split('.'))
        print (f"Updated to openLCA process DB : {process.name} :: {process.id}")        
    else : 
        print (f"Exported to openLCA process DB : {process.name} :: {process.id}")


    process.version = f"{M:02d}.{m:02d}.{(v+1):03d}"
    process.last_change = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    client.put(process)
    Dic_ID_process[process.name] = process.id
    
    return process


Progress_list = []
for Process in Process_list : 
    create_process(Process['Name'],Process['Description'],Process['Tags'],Process['Ex.name'],Process['Ex.amount'],Process['Ex.provider'],Process['Ex.uncertainty'],Progress_list)
    Progress_list.append(Process['Name'])




Updated to openLCA process DB : Battery :: ed53308d-d0fd-4d4b-85ba-896f2e1afbe0
Updated to openLCA process DB : Battery :: ed53308d-d0fd-4d4b-85ba-896f2e1afbe0
Updated to openLCA process DB : Battery :: ed53308d-d0fd-4d4b-85ba-896f2e1afbe0
Updated to openLCA process DB : Control Unit :: 58fd6ee5-43f8-4814-b188-d6b05a36158b
Updated to openLCA process DB : Control Unit :: 58fd6ee5-43f8-4814-b188-d6b05a36158b
Updated to openLCA process DB : Control Unit :: 58fd6ee5-43f8-4814-b188-d6b05a36158b
Updated to openLCA process DB : Harness 4 Power :: 367ab989-fa3b-4bdd-a31e-3989f0907fe3
Updated to openLCA process DB : Harness 4 Power :: 367ab989-fa3b-4bdd-a31e-3989f0907fe3
Updated to openLCA process DB : Power System Assembly & Qualification :: b1073cea-d2f9-47f7-9f53-88d3cc70b72d
Updated to openLCA process DB : Power System Assembly & Qualification :: b1073cea-d2f9-47f7-9f53-88d3cc70b72d
Updated to openLCA process DB : Power System Assembly & Qualification :: b1073cea-d2f9-47f7-9f53-88d3cc70b72d